<a href="https://colab.research.google.com/github/abdulrehmanra0/KeyWord-Extraction/blob/main/KEYWORD_EXTRACTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install dependencies

In [1]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 11.2 MB/s eta 0:00:00


Import dependencies

In [2]:
import json
import pandas as pd
import numpy as np

# Import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F

# Import SparkNLP
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *


In [3]:
# Start Spark session
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


## 2. Inputs

Enter inputs as strings in this list.


In [4]:
input_list = [
    """If AI enables computers to think, computer vision enables them to see, observe and understand.

Computer vision works much the same as human vision, except humans have a head start. Human sight has the advantage of lifetimes of context to train how to tell objects apart, how far away they are, whether they are moving or something is wrong with an image.

Computer vision trains machines to perform these functions, but it must do it in much less time with cameras, data and algorithms rather than retinas, optic nerves and a visual cortex. Because a system trained to inspect products or watch a production asset can analyze thousands of products or processes a minute, noticing imperceptible defects or issues, it can quickly surpass human capabilities.

Computer vision is used in industries that range from energy and utilities to manufacturing and automotive—and the market is continuing to grow. It is expected to reach USD 48.6 billion by 2022.1

Guide
Delve into our exclusive guide to the EU's CSRD
With ESG disclosures starting as early as 2025 for some companies, make sure that you’re prepared with our guide.

Related content
Register for the playbook on smarter asset management

""",

    """What is NLP?
Natural language processing, or NLP, combines computational linguistics—rule-based modeling of human language—with statistical and machine learning models to enable computers and digital devices to recognize, understand and generate text and speech.

A branch of artificial intelligence (AI), NLP lies at the heart of applications and devices that can

translate text from one language to another
respond to typed or spoken commands
recognize or authenticate users based on voice
summarize large volumes of text
assess the intent or sentiment of text or speech
generate text or graphics or other content on demand
often in real time. Today most people have interacted with NLP in the form of voice-operated GPS systems, digital assistants, speech-to-text dictation software, customer service chatbots, and other consumer conveniences. But NLP also plays a growing role in enterprise solutions that help streamline and automate business operations, increase employee productivity, and simplify mission-critical business processes.""",

    """The Prague Quadrennial of Performance Design and Space was established in 1967 to bring the best of design for performance, scenography, and theatre architecture to the front line of cultural activities to be experienced by professional and emerging artists as well as the general public. The quadrennial exhibitions, festivals, and educational programs act as a global catalyst of creative progress by encouraging experimentation, networking, innovation, and future collaborations. PQ aims to honor, empower and celebrate the work of designers, artists and architects while inspiring and educating audiences, who are the most essential element of any live performance. The Prague Quadrennial strives to present performance design as an art form concerned with creation of active performance environments, that are far beyond merely decorative or beautiful, but emotionally charged, where design can become a quest, a question, an argument, a threat, a resolution, an agent of change, or a provocation. Performance design is a collaborative field where designers mix, fuse and blur the lines between multiple artistic disciplines to search for new approaches and new visions.

The Prague Quadrennial organizes an expansive program of international projects and activities between the main quadrennial events – performances, exhibitions, symposia, workshops, residencies, and educational initiatives serve as an international platform for exploring the practice, theory and education of contemporary performance design in the most encompassing terms.""",

    """AWhat is AI?
Artificial intelligence, or AI, is technology that enables computers and machines to simulate human intelligence and problem-solving capabilities.

On its own or combined with other technologies (e.g., sensors, geolocation, robotics) AI can perform tasks that would otherwise require human intelligence or intervention. Digital assistants, GPS guidance, autonomous vehicles, and generative AI tools (like Open AI's Chat GPT) are just a few examples of AI in the daily news and our daily lives.

As a field of computer science, artificial intelligence encompasses (and is often mentioned together with) machine learning and deep learning. These disciplines involve the development of AI algorithms, modeled after the decision-making processes of the human brain, that can ‘learn’ from available data and make increasingly more accurate classifications or predictions over time.

Artificial intelligence has gone through many cycles of hype, but even to skeptics, the release of ChatGPT seems to mark a turning point. The last time generative AI loomed this large, the breakthroughs were in computer vision, but now the leap forward is in natural language processing (NLP). Today, generative AI can learn and synthesize not just human language but other data types including images, video, software code, and even molecular structures.

Applications for AI are growing every day. But as the hype around the use of AI tools in business takes off, conversations around ai ethics and responsible ai become critically important. For more on where IBM stands on these issues, please read Building trust in AI.""",

    """North Ingria was located in the Karelian Isthmus, between Finland and Soviet Russia. It was established 23 January 1919. The republic was first served by a post office at the Rautu railway station on the Finnish side of the border. As the access across the border was mainly restricted, the North Ingrian postal service was finally launched in the early 1920. The man behind the idea was the lieutenant colonel Georg Elfvengren, head of the governing council of North Ingria. He was also known as an enthusiastic stamp collector. The post office was opened at the capital village of Kirjasalo.

The first series of North Ingrian stamps were issued in 21 March 1920. They were based on the 1917 Finnish "Model Saarinen" series, a stamp designed by the Finnish architect Eliel Saarinen. The first series were soon sold to collectors, as the postage stamps became the major financial source of the North Ingrian government. The second series was designed for the North Ingrian postal service and issued 2 August 1920. The value of both series was in Finnish marks and similar to the postal fees of Finland. The number of letters sent from North Ingria was about 50 per day, most of them were carried to Finland. They were mainly sent by the personnel of the Finnish occupying forces. Large number of letters were also sent in pure philatelic purposes.

With the Treaty of Tartu, the area was re-integrated into Soviet Russia and the use of the North Ingrian postage stamps ended in 4 December 1920. Stamps were still sold in Finland in 1921 with an overprinting "Inkerin hyväksi" (For the Ingria), but they were no longer valid. Funds of the sale went for the North Ingrian refugees."""
]

## 3. Pipeline creation

In [5]:
# Transforms the raw text into a document readable by the later stages of the
# pipeline
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

# Separates the document into sentences
sentence_detector = SentenceDetector() \
    .setInputCols(['document']) \
    .setOutputCol('sentences')# \
    #.setDetectLists(True)

# Separates sentences into individial tokens (words)
tokenizer = Tokenizer() \
    .setInputCols(['sentences']) \
    .setOutputCol('tokens') \
    .setContextChars(['(', ')', '?', '!', '.', ','])

# The keyphrase extraction model. Change MinNGrams and MaxNGrams to set the
# minimum and maximum length of possible keyphrases, and change NKeywords to
# set the amount of potential keyphrases identified per document.
keywords = YakeKeywordExtraction() \
    .setInputCols('tokens') \
    .setOutputCol('keywords') \
    .setMinNGrams(2) \
    .setMaxNGrams(5) \
    .setNKeywords(100) \
    .setStopWords(StopWordsCleaner().getStopWords())

# Assemble all of these stages into a pipeline, then fit the pipeline on an
# empty data frame so it can be used to transform new inputs.
pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    keywords
])

pipeline_model = pipeline.fit(spark.createDataFrame([[""]]).toDF('text'))

# LightPipeline is faster than Pipeline for small datasets
light_pipeline = LightPipeline(pipeline_model)

## 4. Result creation

In [6]:
def adjusted_score(row, pow=2.5):
    """This function adjusts the scores of potential key phrases to give better
    scores to phrases with more words (which will naturally have worse scores
    due to the nature of the model). You can change the exponent to reward
    longer phrases more or less. Higher exponents reward longer phrases."""
    return ((row.result.count(' ') + 1) ** pow /
            (float(row.metadata['score']) + 0.1))

def get_top_ranges(phrases, input_text):
    """Combine phrases that overlap."""
    starts = sorted([row['begin'] for row in phrases])
    ends = sorted([row['end'] for row in phrases])

    ranges = [[starts[0], None]]
    for i in range(len(starts) - 1):
        if ends[i] < starts[i + 1]:
            ranges[-1][1] = ends[i]
            ranges.append([starts[i + 1], None])
    ranges[-1][1] = ends[-1]
    return [{
        'begin': range[0],
        'end': range[1],
        'phrase': input_text[4][range[0]:range[1] + 1]  # [4] for last text
     } for range in ranges]

def remove_duplicates(phrases):
    """Remove phrases that appear multiple times."""
    i = 0
    while i < len(phrases):
        j = i + 1
        while j < len(phrases):
            if phrases[i]['phrase'] == phrases[j]['phrase']:
                phrases.remove(phrases[j])
            j += 1
        i += 1

    return phrases

def get_output_lists(df_row):
    """Returns a tuple with two lists of five phrases each. The first combines
    key phrases that overlap to create longer kep phrases, which is best for
    highlighting key phrases in text, and the seocnd is simply the keyphrases
    with the highest scores, which is best for summarizing a document."""
    keyphrases = []
    for row in df_row.keywords[4]:      # [4] for last text
        keyphrases.append({
            'begin': row.begin,
            'end': row.end,
            'phrase': row.result,
            'score': adjusted_score(row)
        })
    keyphrases = sorted(keyphrases, key=lambda x: x['score'], reverse=True)

    return (
        get_top_ranges(keyphrases[:20], df_row.text)[:5],
        remove_duplicates(keyphrases[:10])[:5]
    )

Transform the example inputs to create a data frame storing the identified keyphrases.

In [7]:
from pyspark.sql import Row

# Convert the list of strings to a list of dictionaries with 'text' key
input_dict_list = [{'text': text} for text in input_list]

# Create a list of Row objects where each Row corresponds to a dictionary in input_dict_list
rows = [Row(**i) for i in input_dict_list]

# Create a DataFrame using the Row objects
df = spark.createDataFrame(rows)

# Now, proceed with the transformation
result = pipeline_model.transform(df).toPandas()


In [8]:
top_ranges, top_summaries =  get_output_lists(result.loc[4:,["keywords","text"]])

## 5. Visualize outputs

The raw pandas data frame containing the outputs

In [9]:
result

,text,document,sentences,tokens,keywords
0,"If AI enables computers to think, computer vis...","[(document, 0, 1194, If AI enables computers t...","[(document, 0, 93, If AI enables computers to ...","[(token, 0, 1, If, {'sentence': '0'}, []), (to...","[(chunk, 3, 12, ai enables, {'sentence': '0', ..."
1,"What is NLP?\nNatural language processing, or ...","[(document, 0, 1041, What is NLP?\nNatural lan...","[(document, 0, 11, What is NLP?, {'sentence': ...","[(token, 0, 3, What, {'sentence': '0'}, []), (...","[(chunk, 13, 28, natural language, {'sentence'..."
2,The Prague Quadrennial of Performance Design a...,"[(document, 0, 1548, The Prague Quadrennial of...","[(document, 0, 287, The Prague Quadrennial of ...","[(token, 0, 2, The, {'sentence': '0'}, []), (t...","[(chunk, 4, 21, prague quadrennial, {'sentence..."
3,"AWhat is AI? \nArtificial intelligence, or AI,...","[(document, 0, 1615, AWhat is AI? \nArtificial...","[(document, 0, 11, AWhat is AI?, {'sentence': ...","[(token, 0, 4, AWhat, {'sentence': '0'}, []), ...","[(chunk, 14, 36, artificial intelligence, {'se..."
4,North Ingria was located in the Karelian Isthm...,"[(document, 0, 1679, North Ingria was located ...","[(document, 0, 83, North Ingria was located in...","[(token, 0, 4, North, {'sentence': '0'}, []), ...","[(chunk, 0, 11, north ingria, {'sentence': '0'..."


The list of the top keyphrases (with overlapping keyphrases merged) for the last example

In [10]:
top_ranges

[{'begin': 0, 'end': 11, 'phrase': 'North Ingria'},
 {'begin': 291, 'end': 318, 'phrase': 'North Ingrian postal service'},
 {'begin': 462, 'end': 473, 'phrase': 'North Ingria'},
 {'begin': 599, 'end': 634, 'phrase': 'first series of North Ingrian stamps'},
 {'begin': 895, 'end': 918, 'phrase': 'North Ingrian government'}]

The list of the best summary kephrases (with duplicates removed) for the last example

In [11]:
top_summaries

[{'begin': 291,
  'end': 310,
  'phrase': 'north ingrian postal',
  'score': 59.43933549489857},
 {'begin': 291,
  'end': 318,
  'phrase': 'north ingrian postal service',
  'score': 55.25119651613854},
 {'begin': 291,
  'end': 303,
  'phrase': 'north ingrian',
  'score': 41.928576798395355},
 {'begin': 895,
  'end': 907,
  'phrase': 'north ingrian',
  'score': 41.928576798395355}]